# Plot and analyze results of MESA low-mass star evolution computations 

#### Read comments at the beginning of each cell to understand what it does

In [1]:
# on an astrohub server use ipympl that enables the interactive features of 
# matplotlib in the Jupyter notebook and in JupyterLab
%pylab ipympl  

# for jupyter classic notebook use
#%pylab nbagg

from nugridpy import mesa as ms
from nugridpy import utils as ut

# begin counting figures
ifig=0
for i in range(0,10000):
    close(i)

Populating the interactive namespace from numpy and matplotlib


In [2]:
#### this cell defines functions that allow to suppress unnecessary output information
import os
from contextlib import contextmanager
@contextmanager
def redirect_stdout(new_target):
    old_target, sys.stdout = sys.stdout, new_target
    try:
        yield new_target
    finally:
        sys.stdout = old_target
def get_devnull():
    #return open(os.devnull, "w")
    return open('log_stuff.txt', "w") #where all the stuff goes you don't want to see
####

In [3]:
# astronomical and physical constants in SI units
from astropy import constants as const
from astropy import units as u

Msun = (const.M_sun).value
Rsun = (const.R_sun).value
Lsun = (const.L_sun).value
GN = (const.G).value  # Newton's constant 
sigma = (const.sigma_sb).value  # 1e3 tarnsforms SI units to sgs units
print ('Msun =',Msun)
print ('Rsun =',Rsun)
print ('Lsun =',Lsun)
print ("Newton's G =",GN)
print ("Stefan-Boltzmann constant =",sigma)

# transform to sgs units
sigma = 1e3*sigma
Rsun = 1e2*Rsun
Msun = 1e3*Msun

print ('Stefan-Boltzmann sigma and solar radius and mass in cgs units:',sigma,Rsun,Msun)

Msun = 1.988409870698051e+30
Rsun = 695700000.0
Lsun = 3.828e+26
Newton's G = 6.6743e-11
Stefan-Boltzmann constant = 5.6703744191844314e-08
Stefan-Boltzmann sigma and solar radius and mass in cgs units: 5.6703744191844314e-05 69570000000.0 1.988409870698051e+33


In [4]:
# path to MESA work directory
# on astrohub/csa server
mesa_work_dir = '/user/scratch14_wendi3/dpa/my_mesa_5329/work_low_mass/' 
# on astrohub/outreach server
#mesa_work_dir = '/user/scratch14_outreach/Pavel/work_low_mass' 

# extension of directory name LOGS
suffix = '' # '_nomix'

In [5]:
# plot nova evolutionary track
sh = ms.history_data(mesa_work_dir+'LOGS'+suffix,clean_starlog=True)
age = sh.get('star_age')
model = sh.get('model_number')
lgL = sh.get('log_L')
lgTeff = sh.get('log_Teff')

# read in surface abundances
xli7surf = sh.get('surface_li7')
xc12surf = sh.get('surface_c12')
xc13surf = sh.get('surface_c13')

c12c13 = np.linspace(0,0,len(xc12surf))

for i in range(len(xc12surf)):
    c12c13[i] = (xc12surf[i]/12.)/(xc13surf[i]/13.)
    
ALi = np.linspace(0,0,len(xli7surf))

for i in range(len(xli7surf)):
    ALi[i] = np.log10(xli7surf[i]/7./0.7) + 12.0

ifig=ifig+1;close(ifig);fig=figure(ifig)
size=8
fig.canvas.layout.height = str(0.9*size)+'in'   # This is a hack to prevent ipympl
fig.canvas.layout.width  = str(1.1*size)+'in'   # to adjust horizontal figure size
sh.hrd()

Requested new history.datasa; create new from history.data
 reading ...100% 



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
# read profiles.index file from path to LOGS directory that contains results of MESA nova computations
f = open(mesa_work_dir+'LOGS'+suffix+'/profiles.index', 'r')

profiles = []

i=0
for line in f:
    if i >= 1:
        profiles.append(int(float(line.split()[0])))
    i += 1
    
f.close()

print ("There are",len(profiles),"profiles for the following models:\n", profiles)

There are 119 profiles for the following models:
 [1800, 1900, 2000, 2100, 2200, 2300, 2400, 2500, 2600, 2700, 2800, 2900, 3000, 3100, 3200, 3300, 3400, 3500, 3600, 3700, 3800, 3900, 4000, 4100, 4200, 4300, 4400, 4500, 4600, 4700, 4800, 4900, 5000, 5100, 5200, 5300, 5400, 5500, 5600, 5700, 5800, 5900, 6000, 6100, 6200, 6300, 6400, 6500, 6600, 6700, 6800, 6900, 7000, 7100, 7200, 7300, 7400, 7500, 7600, 7700, 7800, 7900, 8000, 8100, 8200, 8300, 8400, 8500, 8600, 8700, 8800, 8900, 9000, 9100, 9200, 9300, 9400, 9500, 9600, 9700, 9800, 9900, 10000, 10100, 10200, 10300, 10400, 10500, 10600, 10700, 10800, 10900, 11000, 11100, 11200, 11300, 11400, 11500, 11600, 11700, 11800, 11900, 12000, 12100, 12200, 12300, 12400, 12500, 12600, 12700, 12800, 12900, 13000, 13100, 13200, 13300, 13400, 13423, 13444]


In [7]:
# select profiles of every_nth_1 model before and every_nth_2 model after cycle_1 along evolutionary track to plot
cycle_1 = 800
every_nth_1 = 8
every_nth_2 = 10

model_sel = []

for i in range(len(profiles)):
    if profiles[i] <= cycle_1:
        if i%every_nth_1 == 0:
            model_sel.append(profiles[i])
    else:
        if i%every_nth_2 == 0:
            model_sel.append(profiles[i]) 

model_sel

[1800, 2800, 3800, 4800, 5800, 6800, 7800, 8800, 9800, 10800, 11800, 12800]

In [8]:
# add models with selected profiles to the above HRD
lgL_plot = []
lgTeff_plot = []
clr = []

j = 0
for mod in model_sel:
    for i in range(len(model)):
        if int(model[i]) == mod:
            print (i,int(model[i]),lgL[i])
            lgL_plot.append(lgL[i])
            lgTeff_plot.append(lgTeff[i])
    plot(lgTeff_plot[j],lgL_plot[j],marker='o',markerfacecolor=ut.linestylecb(j)[2],\
         markeredgecolor=ut.linestylecb(j)[2],label='model '+str(mod))
    j += 1

xlabel('$\log_{10}\,T_\mathrm{eff}$')
ylabel('$\log_{10}\,L/L_\odot$')
xlim()
legend(frameon=False,loc=2,fontsize=8)
show()

#plt.savefig(mesa_work_dir+'track.pdf')

99 1800 1.6580795687106604
1099 2800 2.1092375534958223
2099 3800 2.423926199549068
3099 4800 2.630704897988842
4099 5800 2.785466440723565
5099 6800 2.9071064840242817
6099 7800 3.0066183722340423
7099 8800 3.0903031046661162
8099 9800 3.1621198971683504
9099 10800 3.2245851503350536
10099 11800 3.2796013017682144
11099 12800 3.328098615321571


In [9]:
# plot radial profiles of various stellar structure parameters

ifig=ifig+1; close(ifig); fig=figure(ifig)
size=8
fig.canvas.layout.height = str(0.9*size)+'in'   # This is a hack to prevent ipympl
fig.canvas.layout.width  = str(1.1*size)+'in'   # to adjust horizontal figure size

omega = 1 # IGW frequency in micro Hz, it will be later scaled by the factor fac_om

# model to plot
model_plot = 2200 # 13400

j = 0

results_dir = mesa_work_dir + 'LOGS'+suffix
mod=ms.mesa_profile(results_dir,model_plot)
rad = Rsun*mod.get('radius') # radius in cm
rad_Mm = 1e-8*rad # radius in Mm
lg_rad = log10(mod.get('radius')) # lg radius in solar units
b_fr = mod.get('brunt_frequency') # in cycles per day
b_nu = mod.get('brunt_nu') # in micro Hz
sl = mod.get('lamb_S') # Lamb frequency for l=1 in Hz
sl = 0.1 * 1e6*sl # Lamb frequency in micro Hz
Mach = 1e2*mod.get('conv_vel_div_csound')
# the following quantities from MESA prifile should be in cgs units
vconv = mod.get('log_conv_vel')
CP = mod.get('cp')
mu = 8*mod.get('mu')
rho = 10**(mod.get('logRho'))
T = 10**(mod.get('logT'))
kappa = 10**(mod.get('log_opacity'))
grad_ad = mod.get('grada')
grad_rad = mod.get('gradr')
conv = grad_rad/grad_ad - 1. # this parameter is used to find all convective boundaries
K = 16*sigma*T**3/(3*kappa*CP*rho**2) # radiative diffusivity in cm**2/s
log_K = log10(K) # logarithm of radiative diffusivity in cm**2/s
f_int = K*(1e-6*b_nu/rad)**3/(1e-6*omega)**4 # the integrand for the IGW "optical depth" tau
Vmax = 10. * 1e2 # cm/s
Pe = log10(Vmax*rad**2*(grad_ad-grad_rad)/(mod.get('pressure_scale_height')*Rsun)/K)
Dmix = mod.get('log_D_mix')
    
plot(lg_rad,b_fr,color=ut.linestylecb(j+1)[2],linestyle=ut.linestylecb(j+1)[0],label='N (cycles per day)')

plot(lg_rad,Mach,color=ut.linestylecb(j+2)[2],linestyle=ut.linestylecb(j+2)[0],label='$100\\times\mathrm{Mach\ number}$')
plot(lg_rad,vconv,color=ut.linestylecb(j+3)[2],linestyle=ut.linestylecb(j+3)[0],label='$\log_{10}\,v_\mathrm{conv}\ (\mathrm{cm\,s}^{-1})$')

#plot(lg_rad,Pe,color=ut.linestylecb(j+4)[2],linestyle=ut.linestylecb(j+4)[0],label='$\log_{10}\,(\mathrm{P\\acute{e}clet\ number}\\times (l(l+1)N^2/\omega^2)$')
plot(lg_rad,Dmix,color=ut.linestylecb(j+5)[2],linestyle=ut.linestylecb(j+5)[0],label='$\log_{10}\,D_\mathrm{mix}$')
plot(lg_rad,sl,color=ut.linestylecb(j+6)[2],linestyle=ut.linestylecb(j+6)[0],label='$0.1\\times\mathrm{Lamb\ frequency}\ S_1$')
plot(lg_rad,log_K,color=ut.linestylecb(j+4)[2],linestyle=ut.linestylecb(j+4)[0],label='$\log_{10}\,K\ (\mathrm{cm}^2\mathrm{s}^{-1})$')
plot(lg_rad,mu,color=ut.linestylecb(j+7)[2],linestyle=ut.linestylecb(j+7)[0],label='$8\\times\\mu$')

# find and plot convective boundaries
rad_conv = []
i_conv = []
for i in range(len(conv)):
    if conv[i]*conv[i-1] <= 0:
        fac = conv[i]/(conv[i]-conv[i-1])
        rad_conv.append((1-fac)*lg_rad[i]+fac*lg_rad[i-1])
        i_conv.append(i)
for i in range(len(rad_conv)):
    vlines(rad_conv[i], -0.5, 20, linestyles ="dashed", colors ="r") 
    
#xlim(2500,15000)
xlim(-2,2)
ylim(-0.5,20)
#xlabel('$r\ (\mathrm{Mm})$',fontsize=14)
xlabel('$\log_{10}\,(r/R_\mathrm{sun})$',fontsize=14)
ylabel('$\mathrm{stellar\ structure\ parameter}$',fontsize=14)
xticks(fontsize=14)
yticks(fontsize=14)
legend(frameon=False,loc=2)
#vlines(2697.6,-1,17,colors='k',linestyles='dotted')
title('model '+str(model_plot))
show()
#savefig('/user/scratch14_wendi3/dpa/M25Envelope.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

119 in profiles.index file ...
Found and load nearest profile for cycle 2200
reading profile/user/scratch14_wendi3/dpa/my_mesa_5329/work_low_mass/LOGS/profile5.data ...
 reading ...100% 



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in log10


In [10]:
# plot molecular weight as a function of radius

ifig=ifig+1; close(ifig); fig=figure(ifig)
size=8
fig.canvas.layout.height = str(0.9*size)+'in'   # This is a hack to prevent ipympl
fig.canvas.layout.width  = str(1.1*size)+'in'   # to adjust horizontal figure size

j = 0

results_dir = mesa_work_dir + 'LOGS'+suffix
mod=ms.mesa_profile(results_dir,model_plot)
rad = Rsun*mod.get('radius') # radius in cm
rad_Mm = 1e-8*rad # radius in Mm
lg_rad = log10(mod.get('radius')) # lg radius in solar units
mu = mod.get('mu')
    
plot(lg_rad,mu,color=ut.linestylecb(j+8)[2],linestyle=ut.linestylecb(j+8)[0],label='')

# find and plot convective boundaries
rad_conv = []
i_conv = []
for i in range(len(conv)):
    if conv[i]*conv[i-1] <= 0:
        fac = conv[i]/(conv[i]-conv[i-1])
        rad_conv.append((1-fac)*lg_rad[i]+fac*lg_rad[i-1])
        i_conv.append(i)
for i in range(len(rad_conv)):
    vlines(rad_conv[i], -0.5, 17, linestyles ="dashed", colors ="r") 
    
xlim(-1.5,0.5)
ymin = 0
ymax = 1.5
ylim(ymin,ymax)
#xlabel('$r\ (\mathrm{Mm})$',fontsize=14)
xlabel('$\log_{10}\,(r/R_\mathrm{sun})$',fontsize=14)
ylabel('$\\mu$',fontsize=14)
xticks(fontsize=14)
yticks(fontsize=14)
#legend(frameon=False,loc=2)
vlines(-1.3,ymin,ymax,colors='k',linestyles='dotted')
title('model '+str(model_plot))
show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

119 in profiles.index file ...
Found and load nearest profile for cycle 2200
reading profile/user/scratch14_wendi3/dpa/my_mesa_5329/work_low_mass/LOGS/profile5.data ...
 reading ...100% 

Closing profile tool ...


In [11]:
# plot abundance profiles as functions of radius

ifig=ifig+1; close(ifig); fig=figure(ifig)
size=8
fig.canvas.layout.height = str(0.9*size)+'in'   # This is a hack to prevent ipympl
fig.canvas.layout.width  = str(1.1*size)+'in'   # to adjust horizontal figure size

j = 0

results_dir = mesa_work_dir + 'LOGS'+suffix
mod=ms.mesa_profile(results_dir,model_plot)
rad = Rsun*mod.get('radius') # radius in cm
rad_Mm = 1e-8*rad # radius in Mm
lg_rad = log10(mod.get('radius')) # lg radius in solar units
xc12 = log10(mod.get('c12'))
xc13 = log10(mod.get('c13'))
xn14 = log10(mod.get('n14'))
xhe3 = log10(mod.get('he3'))
xbe7 = log10(mod.get('be7'))
xli7 = log10(mod.get('li7'))
    
plot(lg_rad,xc12,color=ut.linestylecb(j+1)[2],linestyle=ut.linestylecb(j+1)[0],label='C-12')
plot(lg_rad,xc13,color=ut.linestylecb(j+2)[2],linestyle=ut.linestylecb(j+2)[0],label='C-13')
plot(lg_rad,xn14,color=ut.linestylecb(j+3)[2],linestyle=ut.linestylecb(j+3)[0],label='N-14')
plot(lg_rad,xhe3,color=ut.linestylecb(j+4)[2],linestyle=ut.linestylecb(j+4)[0],label='He-3')
plot(lg_rad,xbe7,color=ut.linestylecb(j+5)[2],linestyle=ut.linestylecb(j+5)[0],label='Be-7')
plot(lg_rad,xli7,color=ut.linestylecb(j+6)[2],linestyle=ut.linestylecb(j+6)[0],label='Li-7')

# find and plot convective boundaries
#rad_conv = []
#i_conv = []
#for i in range(len(conv)):
#    if conv[i]*conv[i-1] <= 0:
#        fac = conv[i]/(conv[i]-conv[i-1])
#        rad_conv.append((1-fac)*lg_rad[i]+fac*lg_rad[i-1])
#        i_conv.append(i)
#for i in range(len(rad_conv)):
#    vlines(rad_conv[i], -12, -2, linestyles ="dashed", colors ="r") 
    
xlim(-1.5,0.5)
ylim(-12,-2)
#xlabel('$r\ (\mathrm{Mm})$',fontsize=14)
xlabel('$\log_{10}\,(r/R_\mathrm{sun})$',fontsize=14)
ylabel('$\log_{10}\,X_i$',fontsize=14)
xticks(fontsize=14)
yticks(fontsize=14)
legend(frameon=False,loc='lower center')
vlines(-1.3,-12,-2,colors='k',linestyles='dotted')
title('model '+str(model_plot))
show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

119 in profiles.index file ...
Found and load nearest profile for cycle 2200
reading profile/user/scratch14_wendi3/dpa/my_mesa_5329/work_low_mass/LOGS/profile5.data ...
 reading ...100% 

Closing profile tool ...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in log10
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in log10


In [12]:
# plot abundance profiles as functions of mass coordinate

ifig=ifig+1; close(ifig); fig=figure(ifig)
size=8
fig.canvas.layout.height = str(0.9*size)+'in'   # This is a hack to prevent ipympl
fig.canvas.layout.width  = str(1.1*size)+'in'   # to adjust horizontal figure size

j = 0

results_dir = mesa_work_dir + 'LOGS'+suffix
mod=ms.mesa_profile(results_dir,model_plot)
mr = mod.get('mass') # radius in cm
xc12 = log10(mod.get('c12'))
xc13 = log10(mod.get('c13'))
xn14 = log10(mod.get('n14'))
xhe3 = log10(mod.get('he3'))
xbe7 = log10(mod.get('be7'))
xli7 = log10(mod.get('li7'))
    
plot(mr,xc12,color=ut.linestylecb(j+1)[2],linestyle=ut.linestylecb(j+1)[0],label='C-12')
plot(mr,xc13,color=ut.linestylecb(j+2)[2],linestyle=ut.linestylecb(j+2)[0],label='C-13')
plot(mr,xn14,color=ut.linestylecb(j+3)[2],linestyle=ut.linestylecb(j+3)[0],label='N-14')
plot(mr,xhe3,color=ut.linestylecb(j+4)[2],linestyle=ut.linestylecb(j+4)[0],label='He-3')
plot(mr,xbe7,color=ut.linestylecb(j+5)[2],linestyle=ut.linestylecb(j+5)[0],label='Be-7')
plot(mr,xli7,color=ut.linestylecb(j+6)[2],linestyle=ut.linestylecb(j+6)[0],label='Li-7')

# find and plot convective boundaries
mr_conv = []
i_conv = []
for i in range(len(conv)):
    if conv[i]*conv[i-1] <= 0:
        fac = conv[i]/(conv[i]-conv[i-1])
        mr_conv.append((1-fac)*mr[i]+fac*mr[i-1])
        i_conv.append(i)
for i in range(len(rad_conv)):
    vlines(mr_conv[i], -12, -2, linestyles ="dashed", colors ="r") 
    
xlim(0.25,1.0)
ylim(-12,-2)
#xlabel('$r\ (\mathrm{Mm})$',fontsize=14)
xlabel('$M_r/M_\odot$',fontsize=14)
ylabel('$\log_{10}\,X_i$',fontsize=14)
xticks(fontsize=14)
yticks(fontsize=14)
legend(frameon=False,loc='lower center')
vlines(-1.3,-12,-2,colors='k',linestyles='dotted')
title('model '+str(model_plot))
show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

119 in profiles.index file ...
Found and load nearest profile for cycle 2200
reading profile/user/scratch14_wendi3/dpa/my_mesa_5329/work_low_mass/LOGS/profile5.data ...
 reading ...100% 

Closing profile tool ...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log10
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in log10


In [13]:
# a plot of surface C12/C13 abundance ratio

ifig=ifig+1; close(ifig); fig=figure(ifig)
size=8
fig.canvas.layout.height = str(0.9*size)+'in'   # This is a hack to prevent ipympl
fig.canvas.layout.width  = str(1.1*size)+'in'   # to adjust horizontal figure size

j = 0
plot(lgL,c12c13,color=ut.linestylecb(j+1)[2],linestyle=ut.linestylecb(j+1)[0],label='')

xlim(1.5,3.5)
ylim(0,30)
xlabel('$\log_{10}\,L/L_\odot$',fontsize=14)
ylabel('$^{12}\mathrm{C}/^{13}\mathrm{C}$',fontsize=14)
xticks(fontsize=14)
yticks(fontsize=14)
show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
# a plot of ALi

ifig=ifig+1; close(ifig); fig=figure(ifig)
size=8
fig.canvas.layout.height = str(0.9*size)+'in'   # This is a hack to prevent ipympl
fig.canvas.layout.width  = str(1.1*size)+'in'   # to adjust horizontal figure size

j = 7
plot(ALi,lgL,color=ut.linestylecb(j+1)[2],linestyle=ut.linestylecb(j+1)[0],label='')

ylim(1.5,3.5)
xlim(3.5,-2)
ylabel('$\log_{10}\,L/L_\odot$',fontsize=14)
xlabel('$A(^7\mathrm{Li})$',fontsize=14)
xticks(fontsize=14)
yticks(fontsize=14)
show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …